In [1]:
2+3

5

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Pandas Ex").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/26 14:06:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pandas as pd

In [4]:
p_df1 = pd.read_csv("DataSets/transactions_1.csv")
p_df1

,cid,cname,dateTime,amount,discount,member
0,101,Niki,2021-07-22 01:01:53,2415.12,8 %,True
1,102,Raaki,2021-08-13 12:15:33,9399.22,10 %,True
2,103,Maaki,2022-08-18 22:12:45,1384.16,20 %,False
3,104,Nibba,2022-07-08 14:18:37,2246.00,9 %,True
4,105,Nibbi,2023-01-04 07:13:13,3145.13,12 %,False


In [6]:
p_df1.dtypes

cid           int64
cname        object
dateTime     object
amount      float64
discount     object
member         bool
dtype: object

In [7]:
s_df1 = spark.createDataFrame(p_df1)
s_df1.show()

/usr/local/lib/python3.8/dist-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+---+-----+-------------------+-------+--------+------+
|cid|cname|           dateTime| amount|discount|member|
+---+-----+-------------------+-------+--------+------+
|101| Niki|2021-07-22 01:01:53|2415.12|     8 %|  true|
|102|Raaki|2021-08-13 12:15:33|9399.22|    10 %|  true|
|103|Maaki|2022-08-18 22:12:45|1384.16|    20 %| false|
|104|Nibba|2022-07-08 14:18:37| 2246.0|     9 %|  true|
|105|Nibbi|2023-01-04 07:13:13|3145.13|    12 %| false|
+---+-----+-------------------+-------+--------+------+



In [8]:
p_df2 = pd.read_csv("DataSets/transactions_2.csv")
p_df2

,cid,cname,dateTime,amount,discount,member
0,106,ANiki,2021-07-22 01:01:53,2415.12,8 %,True
1,107,BRaaki,2021-08-13 12:15:33,9399.22,10 %,True
2,108,CMaaki,2022-08-18 22:12:45,1384.16,NaN,False
3,109,DNibba,2022-07-08 14:18:37,2246.00,9 %,True
4,110,ENibbi,NaN,3145.13,12 %,False


In [9]:
p_df2.dtypes

cid           int64
cname        object
dateTime     object
amount      float64
discount     object
member         bool
dtype: object

In [10]:
s_df2 = spark.createDataFrame(p_df2)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


TypeError: field discount: Can not merge type <class 'pyspark.sql.types.StringType'> and <class 'pyspark.sql.types.DoubleType'>

In [11]:
s_df2 = spark.createDataFrame(p_df2.astype('str'))
s_df2.show()

+---+------+-------------------+-------+--------+------+
|cid| cname|           dateTime| amount|discount|member|
+---+------+-------------------+-------+--------+------+
|106| ANiki|2021-07-22 01:01:53|2415.12|     8 %|  True|
|107|BRaaki|2021-08-13 12:15:33|9399.22|    10 %|  True|
|108|CMaaki|2022-08-18 22:12:45|1384.16|     nan| False|
|109|DNibba|2022-07-08 14:18:37| 2246.0|     9 %|  True|
|110|ENibbi|                nan|3145.13|    12 %| False|
+---+------+-------------------+-------+--------+------+



Otherwise, we can convert pandas df to spark df with our own Schema

In [12]:
from pyspark.sql.types import StructField,StructType,BooleanType,FloatType,IntegerType,StringType

own_schema = StructType([
    StructField("Cid",IntegerType(),True),
    StructField("CName",StringType(),True),
    StructField("DateTime",StringType(),True),
    StructField("Amount",FloatType(),True),
    StructField("Discount",StringType(),True),
    StructField("Member",BooleanType(),True)
])

s_df2 = spark.createDataFrame(p_df2,schema=own_schema)
s_df2.show()

+---+------+-------------------+-------+--------+------+
|Cid| CName|           DateTime| Amount|Discount|Member|
+---+------+-------------------+-------+--------+------+
|106| ANiki|2021-07-22 01:01:53|2415.12|     8 %|  true|
|107|BRaaki|2021-08-13 12:15:33|9399.22|    10 %|  true|
|108|CMaaki|2022-08-18 22:12:45|1384.16|     NaN| false|
|109|DNibba|2022-07-08 14:18:37| 2246.0|     9 %|  true|
|110|ENibbi|                NaN|3145.13|    12 %| false|
+---+------+-------------------+-------+--------+------+



In [13]:
s_df2.printSchema()

root
 |-- Cid: integer (nullable = true)
 |-- CName: string (nullable = true)
 |-- DateTime: string (nullable = true)
 |-- Amount: float (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Member: boolean (nullable = true)

